In [1]:
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import os
import sys
import itertools
import matplotlib.pyplot as plt
import seaborn as sns

# Base Case Summer
### Loading Data

In [3]:
# Get the absolute path to the 'src' directory
project_root = os.path.abspath(os.path.join(os.getcwd(), "..")) #change when running in a different directory
src_path = os.path.join(project_root, "src")

# Add 'src' to system path
if src_path not in sys.path:
    sys.path.append(src_path)

from create_Dataframe import createDataframe as create_df

merged_data_summer = create_df('summer')

/Users/simonbernet/Desktop/Optimization in Energy Systems/Optimization Project/optimization_project/src/create_Dataframe.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pv_summer["timestamp"] = pd.to_datetime(
/Users/simonbernet/Desktop/Optimization in Energy Systems/Optimization Project/optimization_project/src/create_Dataframe.py:110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pv_winter["timestamp"] = pd.to_datetime(


### Global Variables

In [ ]:
price = merged_data_summer["Spotmarket_(EUR/kWh)"].values
inflexible_demand = merged_data_summer['Inflexible_Demand_(kWh)'].values
Time_interval = len(price)  # Total time interval in hours

# Initiate gurobi model
model = gp.Model("automated_demand_response_base_case_summer")
model.update()

power_dishwasher = 1.5
power_wm = 3
power_dryer = 3
max_power_ev = 10
max_power_hp = 0 #no heatpump in this case

Set parameter Username
Set parameter LicenseID to value 2653942
Academic license - for non-commercial use only - expires 2026-04-17


### Appliances

In [ ]:
from cases import *

binary_dishwasher, dishwasher_start, start_times = dishwasher(Time_interval,merged_data_summer,model)
duration_wm, wm_start, binary_wm, start_times_wm = washing_machine(Time_interval,merged_data_summer,model)
binary_dryer, start_times_dryer, dryer_start = dryer(Time_interval,merged_data_summer,model, duration_wm, wm_start)
charging_ev, soc_ev, binary_ev = EV_no_feed_in(Time_interval,merged_data_summer,model)
penalty_cost, level_bin, levels, penalty_per_level, total_demand = peak_prices(Time_interval,merged_data_summer,model,
            inflexible_demand, binary_wm, binary_dishwasher, binary_dryer, charging_ev, power_wm, 
            power_dishwasher, max_power_ev, power_dryer, max_power_hp)



### Run Optimization

In [6]:
# Electricity cost
electricity_cost = gp.quicksum(
    (price[t] *
        (merged_data_summer['Inflexible_Demand_(kWh)'][t] +
        power_dishwasher * binary_dishwasher[t] +
        power_wm * binary_wm[t] +
        power_dryer * binary_dryer[t] +
        charging_ev[t])) for t in range(Time_interval)
)
# Full objective: cost + penalty
model.setObjective(electricity_cost + penalty_cost, GRB.MINIMIZE)

# Optimize
model.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[x86] - Darwin 21.6.0 21H1320)

CPU model: Intel(R) Core(TM) i5-6267U CPU @ 2.90GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 20856 rows, 10078 columns and 58541 nonzeros


Model fingerprint: 0x73501b80
Variable types: 2016 continuous, 8062 integer (8062 binary)
Coefficient statistics:
  Matrix range     [2e-01, 3e+01]
  Objective range  [2e-03, 7e-01]
  Bounds range     [1e+00, 7e+01]
  RHS range        [7e-02, 7e+01]
Presolve removed 15414 rows and 5051 columns
Presolve time: 0.44s
Presolved: 5442 rows, 5027 columns, 28511 nonzeros
Variable types: 680 continuous, 4347 integer (4347 binary)

Root relaxation: objective 3.905795e+01, 2700 iterations, 0.08 seconds (0.03 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   39.05795    0  225          -   39.05795      -     -    0s
H    0     0                      62.4133989   39.05795  37.4%     -    0s
H    0     0                      61.2247689   39.05795  36.2%     -    0s
H    0     0                      51.9861690   39.05795  24.9%     -    0s
H    0     0                    

### Save Data

In [7]:
results_df = merged_data_summer.copy()
results_df['Total_Demand'] = [total_demand[t].X for t in range(len(results_df))]
results_df['Dishwasher_Start'] = [dishwasher_start[t].X if t in start_times else 0 for t in range(len(results_df))]
results_df['Dishwasher_On'] = [binary_dishwasher[t].X for t in range(len(results_df))]
results_df['Washing_Machine_Start'] = [wm_start[t].X if t in start_times_wm else 0 for t in range(len(results_df))]
results_df['Washing_Machine_On'] = [binary_wm[t].X for t in range(len(results_df))]
results_df['Dryer_Start'] = [dryer_start[t].X if t in start_times_dryer else 0 for t in range(len(results_df))]
results_df['Dryer_On'] = [binary_dryer[t].X for t in range(len(results_df))]
results_df['EV_SOC'] = [soc_ev[t].X for t in range(len(results_df))]
results_df['EV_Charging'] = [charging_ev[t].X for t in range(len(results_df))]
results_df['EV_On'] = [binary_ev[t].X for t in range(len(results_df))]
# Add demand level index for each hour directly from the Gurobi binary variables
results_df['Demand_Level'] = [
    int(np.argmax([level_bin[t][i].X for i in range(len(levels)-1)]))
    for t in range(len(results_df))
]

# Add penalty cost per hour based on demand level
results_df['Penalty_Cost'] = [
    penalty_per_level[dl] for dl in results_df['Demand_Level']
]

# Add electricity cost per hour
results_df['Electricity_Cost'] = results_df['Total_Demand'] * results_df['Spotmarket_(EUR/kWh)']

# Add total cost per hour (electricity + penalty)
results_df['Total_Cost'] = results_df['Electricity_Cost'] + results_df['Penalty_Cost']

# Save to CSV
results_csv_path = os.path.join(project_root, "results_basecase_summer.csv")
results_df.to_csv(results_csv_path, index=False)
print(f"Results saved to {results_csv_path}")

Results saved to /Users/simonbernet/Desktop/Optimization in Energy Systems/Optimization Project/optimization_project/results_basecase_summer.csv


# Base Case Winter
### Loading Data

In [2]:
# Get the absolute path to the 'src' directory
project_root = os.path.abspath(os.path.join(os.getcwd(), "..")) #change when running in a different directory
src_path = os.path.join(project_root, "src")

# Add 'src' to system path
if src_path not in sys.path:
    sys.path.append(src_path)

from create_Dataframe import createDataframe as create_df

merged_data_winter = create_df('winter')

/Users/simonbernet/Desktop/Optimization in Energy Systems/Optimization Project/optimization_project/src/create_Dataframe.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pv_summer["timestamp"] = pd.to_datetime(
/Users/simonbernet/Desktop/Optimization in Energy Systems/Optimization Project/optimization_project/src/create_Dataframe.py:110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pv_winter["timestamp"] = pd.to_datetime(


### Global Variables

In [ ]:
price = merged_data_winter["Spotmarket_(EUR/kWh)"].values
inflexible_demand = merged_data_winter['Inflexible_Demand_(kWh)'].values
Time_interval = len(price)  # Total time interval in hours

# Initiate gurobi model
model = gp.Model("automated_demand_response_base_case_winter")
model.update()

power_dishwasher = 1.5
power_wm = 3
power_dryer = 3
max_power_ev = 10
max_power_hp = 0 #no heatpump in this case

Set parameter Username
Set parameter LicenseID to value 2653942
Academic license - for non-commercial use only - expires 2026-04-17


### Appliances

In [ ]:
from cases import *

binary_dishwasher, dishwasher_start, start_times = dishwasher(Time_interval,merged_data_winter,model)
duration_wm, wm_start, binary_wm, start_times_wm = washing_machine(Time_interval,merged_data_winter,model)
binary_dryer, start_times_dryer, dryer_start = dryer(Time_interval,merged_data_winter,model, duration_wm, wm_start)
charging_ev, soc_ev, binary_ev = EV_no_feed_in(Time_interval,merged_data_winter,model)
penalty_cost, level_bin, levels, penalty_per_level, total_demand = peak_prices(Time_interval,merged_data_winter,model,
            inflexible_demand, binary_wm, binary_dishwasher, binary_dryer, charging_ev, power_wm, 
            power_dishwasher, max_power_ev, power_dryer, max_power_hp)

### Objective Function and Optimization

In [5]:
# Electricity cost
electricity_cost = gp.quicksum(
    (price[t] *
        (merged_data_winter['Inflexible_Demand_(kWh)'][t] +
        power_dishwasher * binary_dishwasher[t] +
        power_wm * binary_wm[t] +
        power_dryer * binary_dryer[t] +
        charging_ev[t])) for t in range(Time_interval)
)
# Full objective: cost + penalty
model.setObjective(electricity_cost + penalty_cost, GRB.MINIMIZE)

# Optimize
model.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[x86] - Darwin 21.6.0 21H1320)

CPU model: Intel(R) Core(TM) i5-6267U CPU @ 2.90GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 20856 rows, 10078 columns and 58541 nonzeros
Model fingerprint: 0xb882c6b3
Variable types: 2016 continuous, 8062 integer (8062 binary)
Coefficient statistics:
  Matrix range     [2e-01, 3e+01]
  Objective range  [2e-04, 7e-01]
  Bounds range     [1e+00, 7e+01]
  RHS range        [9e-02, 7e+01]
Presolve removed 15421 rows and 5057 columns
Presolve time: 1.31s
Presolved: 5435 rows, 5021 columns, 28497 nonzeros
Variable types: 678 continuous, 4343 integer (4343 binary)

Root relaxation: objective 4.522003e+01, 2688 iterations, 0.19 seconds (0.03 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   45.22003    0  167          -   45

### Save Data

In [ ]:
results_df = merged_data_winter.copy()
results_df['Total_Demand'] = [total_demand[t].X for t in range(len(results_df))]
results_df['Dishwasher_Start'] = [dishwasher_start[t].X if t in start_times else 0 for t in range(len(results_df))]
results_df['Dishwasher_On'] = [binary_dishwasher[t].X for t in range(len(results_df))]
results_df['Washing_Machine_Start'] = [wm_start[t].X if t in start_times_wm else 0 for t in range(len(results_df))]
results_df['Washing_Machine_On'] = [binary_wm[t].X for t in range(len(results_df))]
results_df['Dryer_Start'] = [dryer_start[t].X if t in start_times_dryer else 0 for t in range(len(results_df))]
results_df['Dryer_On'] = [binary_dryer[t].X for t in range(len(results_df))]
results_df['EV_SOC'] = [soc_ev[t].X for t in range(len(results_df))]
results_df['EV_Charging'] = [charging_ev[t].X for t in range(len(results_df))]
results_df['EV_On'] = [binary_ev[t].X for t in range(len(results_df))]
# Add demand level index for each hour directly from the Gurobi binary variables
results_df['Demand_Level'] = [
    int(np.argmax([level_bin[t][i].X for i in range(len(levels)-1)]))
    for t in range(len(results_df))
]

# Add penalty cost per hour based on demand level
results_df['Penalty_Cost'] = [
    penalty_per_level[dl] for dl in results_df['Demand_Level']
]

# Add electricity cost per hour
results_df['Electricity_Cost'] = results_df['Total_Demand'] * results_df['Spotmarket_(EUR/kWh)']

# Add total cost per hour (electricity + penalty)
results_df['Total_Cost'] = results_df['Electricity_Cost'] + results_df['Penalty_Cost']

# Save to CSV
results_csv_path = os.path.join(project_root, "results_basecase_winter.csv")
results_df.to_csv(results_csv_path, index=False)
print(f"Results saved to {results_csv_path}")

# Case 1 Winter - with heatpump 
### Loading Data

In [2]:
# Get the absolute path to the 'src' directory
project_root = os.path.abspath(os.path.join(os.getcwd(), "..")) #change when running in a different directory
src_path = os.path.join(project_root, "src")

# Add 'src' to system path
if src_path not in sys.path:
    sys.path.append(src_path)

from create_Dataframe import createDataframe as create_df

merged_data_winter_case_1 = create_df('winter')

/Users/simonbernet/Desktop/Optimization in Energy Systems/Optimization Project/optimization_project/src/create_Dataframe.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pv_summer["timestamp"] = pd.to_datetime(
/Users/simonbernet/Desktop/Optimization in Energy Systems/Optimization Project/optimization_project/src/create_Dataframe.py:110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pv_winter["timestamp"] = pd.to_datetime(


### Global Variables

In [ ]:
price = merged_data_winter_case_1["Spotmarket_(EUR/kWh)"].values
inflexible_demand = merged_data_winter_case_1['Inflexible_Demand_(kWh)'].values
Time_interval = len(price)  # Total time interval in hours

# Initiate gurobi model
model = gp.Model("automated_demand_response_case_1_winter")
model.update()

power_dishwasher = 1.5
power_wm = 3
power_dryer = 3
max_power_ev = 10
max_power_hp = 8

Set parameter Username
Set parameter LicenseID to value 2653942
Academic license - for non-commercial use only - expires 2026-04-17


### Appliances

In [ ]:
from cases import *

binary_dishwasher, dishwasher_start, start_times = dishwasher(Time_interval,merged_data_winter_case_1,model)
duration_wm, wm_start, binary_wm, start_times_wm = washing_machine(Time_interval,merged_data_winter_case_1,model)
binary_dryer, start_times_dryer, dryer_start = dryer(Time_interval,merged_data_winter_case_1,model, duration_wm, wm_start)
charging_ev, soc_ev, binary_ev = EV_no_feed_in(Time_interval,merged_data_winter_case_1,model)
power_hp = heat_pump(Time_interval,merged_data_winter_case_1,model,max_power_hp)

penalty_cost, level_bin, levels, penalty_per_level, total_demand = peak_prices(Time_interval,merged_data_winter_case_1,model,
            inflexible_demand, binary_wm, binary_dishwasher, binary_dryer, charging_ev, power_wm, 
            power_dishwasher, max_power_ev, power_dryer, max_power_hp,power_hp)


In [5]:
# Electricity cost
electricity_cost = gp.quicksum(
    (price[t] *
        (merged_data_winter_case_1['Inflexible_Demand_(kWh)'][t] +
        power_dishwasher * binary_dishwasher[t] +
        power_wm * binary_wm[t] +
        power_dryer * binary_dryer[t] +
        charging_ev[t] +
        power_hp[t])) for t in range(Time_interval)
)
# Full objective: cost + penalty
model.setObjective(electricity_cost + penalty_cost, GRB.MINIMIZE)

# Optimize
model.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[x86] - Darwin 21.6.0 21H1320)

CPU model: Intel(R) Core(TM) i5-6267U CPU @ 2.90GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 22200 rows, 12094 columns and 61899 nonzeros
Model fingerprint: 0x74583b72
Variable types: 4032 continuous, 8062 integer (8062 binary)
Coefficient statistics:
  Matrix range     [2e-01, 4e+01]
  Objective range  [2e-04, 7e-01]
  Bounds range     [1e+00, 2e+02]
  RHS range        [5e-05, 1e+02]
Presolve removed 17222 rows and 7883 columns
Presolve time: 0.87s
Presolved: 4978 rows, 4211 columns, 27350 nonzeros
Variable types: 674 continuous, 3537 integer (3537 binary)
Found heuristic solution: objective 76.4943101

Root relaxation: objective 5.038396e+01, 2003 iterations, 0.04 seconds (0.02 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

  

### Save Data

In [ ]:
results_df = merged_data_winter_case_1.copy()
results_df['Total_Demand'] = [total_demand[t].X for t in range(len(results_df))]
results_df['Dishwasher_Start'] = [dishwasher_start[t].X if t in start_times else 0 for t in range(len(results_df))]
results_df['Dishwasher_On'] = [binary_dishwasher[t].X for t in range(len(results_df))]
results_df['Washing_Machine_Start'] = [wm_start[t].X if t in start_times_wm else 0 for t in range(len(results_df))]
results_df['Washing_Machine_On'] = [binary_wm[t].X for t in range(len(results_df))]
results_df['Dryer_Start'] = [dryer_start[t].X if t in start_times_dryer else 0 for t in range(len(results_df))]
results_df['Dryer_On'] = [binary_dryer[t].X for t in range(len(results_df))]
results_df['EV_SOC'] = [soc_ev[t].X for t in range(len(results_df))]
results_df['EV_Charging'] = [charging_ev[t].X for t in range(len(results_df))]
results_df['EV_On'] = [binary_ev[t].X for t in range(len(results_df))]
# Add demand level index for each hour directly from the Gurobi binary variables
results_df['Demand_Level'] = [
    int(np.argmax([level_bin[t][i].X for i in range(len(levels)-1)]))
    for t in range(len(results_df))
]

# Add penalty cost per hour based on demand level
results_df['Penalty_Cost'] = [
    penalty_per_level[dl] for dl in results_df['Demand_Level']
]

# Add electricity cost per hour
results_df['Electricity_Cost'] = results_df['Total_Demand'] * results_df['Spotmarket_(EUR/kWh)']

# Add total cost per hour (electricity + penalty)
results_df['Total_Cost'] = results_df['Electricity_Cost'] + results_df['Penalty_Cost']

# Add heatpump data
#
#
#


# Save to CSV
results_csv_path = os.path.join(project_root, "results_case_1_winter.csv")
results_df.to_csv(results_csv_path, index=False)
print(f"Results saved to {results_csv_path}")

# Case 2 Summer 
## PV Generation without feed-in
### Loading Data

In [2]:
# Get the absolute path to the 'src' directory
project_root = os.path.abspath(os.path.join(os.getcwd(), "..")) #change when running in a different directory
src_path = os.path.join(project_root, "src")

# Add 'src' to system path
if src_path not in sys.path:
    sys.path.append(src_path)

from create_Dataframe import createDataframe as create_df

merged_data_summer_case_2 = create_df('summer')

/Users/simonbernet/Desktop/Optimization in Energy Systems/Optimization Project/optimization_project/src/create_Dataframe.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pv_summer["timestamp"] = pd.to_datetime(
/Users/simonbernet/Desktop/Optimization in Energy Systems/Optimization Project/optimization_project/src/create_Dataframe.py:110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pv_winter["timestamp"] = pd.to_datetime(


### Global Variables

In [3]:
price = merged_data_summer_case_2["Spotmarket_(EUR/kWh)"].values
inflexible_demand = merged_data_summer_case_2['Inflexible_Demand_(kWh)'].values
Time_interval = len(price)  # Total time interval in hours

# Initiate gurobi model
model = gp.Model("automated_demand_response_case_2_summer")
model.update()

power_dishwasher = 1.5
power_wm = 3
power_dryer = 3
max_power_ev = 10
max_power_hp = 0 #summer -> heatpump not in use
power_hp = [0] * Time_interval #summer -> heatpump not in use

Set parameter Username
Set parameter LicenseID to value 2653942
Academic license - for non-commercial use only - expires 2026-04-17


### Appliances

In [4]:
from cases import *

binary_dishwasher, dishwasher_start, start_times = dishwasher(Time_interval,merged_data_summer_case_2,model)
duration_wm, wm_start, binary_wm, start_times_wm = washing_machine(Time_interval,merged_data_summer_case_2,model)
binary_dryer, start_times_dryer, dryer_start = dryer(Time_interval,merged_data_summer_case_2,model, duration_wm, wm_start)
charging_ev, soc_ev, binary_ev = EV_no_feed_in(Time_interval,merged_data_summer_case_2,model)
pv, load, unmet, curtail, pv_maxed_binary, total_demand,penalty_cost, level_bin, levels, penalty_per_level, demand_level= PV_no_feed_in_and_penalty(Time_interval,
                            merged_data_summer_case_2,model,merged_data_summer_case_2,power_dishwasher,binary_dishwasher
                            ,power_wm,binary_wm,power_dryer,binary_dryer,charging_ev,inflexible_demand,max_power_ev,max_power_hp,power_hp)


### Objective Function

In [5]:
# Electricity cost
electricity_cost = gp.quicksum(price[t] * unmet[t] for t in range(Time_interval))

# Full objective: cost + penalty
model.setObjective(electricity_cost + penalty_cost, GRB.MINIMIZE)

# Optimize
model.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[x86] - Darwin 21.6.0 21H1320)

CPU model: Intel(R) Core(TM) i5-6267U CPU @ 2.90GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 22872 rows, 12094 columns and 65261 nonzeros
Model fingerprint: 0x33bfa741
Variable types: 2688 continuous, 9406 integer (8734 binary)
Coefficient statistics:
  Matrix range     [2e-01, 1e+04]
  Objective range  [2e-03, 2e-01]
  Bounds range     [1e+00, 7e+01]
  RHS range        [1e-04, 1e+04]
Presolve removed 17042 rows and 6683 columns
Presolve time: 0.41s
Presolved: 5830 rows, 5411 columns, 28665 nonzeros
Variable types: 896 continuous, 4515 integer (4515 binary)

Root relaxation: objective 2.200795e+01, 3318 iterations, 0.09 seconds (0.05 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   22.00795    0  384          -   22

### Save Data 

In [7]:
# Prepare results DataFrame
results_df = merged_data_summer_case_2.copy()
results_df['Total_Demand'] = [total_demand[t].getValue() for t in range(len(results_df))]
results_df['Dishwasher_Start'] = [dishwasher_start[t].X if t in start_times else 0 for t in range(len(results_df))]
results_df['Dishwasher_On'] = [binary_dishwasher[t].X for t in range(len(results_df))]
results_df['Washing_Machine_Start'] = [wm_start[t].X if t in start_times_wm else 0 for t in range(len(results_df))]
results_df['Washing_Machine_On'] = [binary_wm[t].X for t in range(len(results_df))]
results_df['Dryer_Start'] = [dryer_start[t].X if t in start_times_dryer else 0 for t in range(len(results_df))]
results_df['Dryer_On'] = [binary_dryer[t].X for t in range(len(results_df))]
results_df['EV_SOC'] = [soc_ev[t].X for t in range(len(results_df))]
results_df['EV_Charging'] = [charging_ev[t].X for t in range(len(results_df))]
results_df['EV_On'] = [binary_ev[t].X for t in range(len(results_df))]
results_df['Unmet'] = [unmet[t].X for t in range(len(results_df))]
results_df['Demand_Level'] = [demand_level[t].X for t in range(len(results_df))]
results_df['Penalty'] = [
    sum(
        penalty_per_level[i] * level_bin[t][i].X
        for i in range(len(penalty_per_level))
    )
    for t in range(len(results_df))
]
results_df['Electricity_Cost'] = [
    price[t] * unmet[t].X for t in range(len(results_df))
]
results_df['Total_Price'] = [
    results_df['Electricity_Cost'][t] + results_df['Penalty'][t]
    for t in range(len(results_df))
]
# Save to CSV
results_csv_path = os.path.join(project_root, "results_case_2_summer.csv")
results_df.to_csv(results_csv_path, index=False)
print(f"Results saved to {results_csv_path}")

Results saved to /Users/simonbernet/Desktop/Optimization in Energy Systems/Optimization Project/optimization_project/results_case_2_summer.csv


# Case 2 Winter 
## PV Generation without feed-in
### Loading Data

In [2]:
# Get the absolute path to the 'src' directory
project_root = os.path.abspath(os.path.join(os.getcwd(), "..")) #change when running in a different directory
src_path = os.path.join(project_root, "src")

# Add 'src' to system path
if src_path not in sys.path:
    sys.path.append(src_path)

from create_Dataframe import createDataframe as create_df

merged_data_winter_case_2 = create_df('winter')

/Users/simonbernet/Desktop/Optimization in Energy Systems/Optimization Project/optimization_project/src/create_Dataframe.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pv_summer["timestamp"] = pd.to_datetime(
/Users/simonbernet/Desktop/Optimization in Energy Systems/Optimization Project/optimization_project/src/create_Dataframe.py:110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pv_winter["timestamp"] = pd.to_datetime(


### Global Variables

In [5]:
price = merged_data_winter_case_2["Spotmarket_(EUR/kWh)"].values
inflexible_demand = merged_data_winter_case_2['Inflexible_Demand_(kWh)'].values
Time_interval = len(price)  # Total time interval in hours

# Initiate gurobi model
model = gp.Model("automated_demand_response_case_2_winter")
model.update()

power_dishwasher = 1.5
power_wm = 3
power_dryer = 3
max_power_ev = 10
max_power_hp = 8


### Appliances


In [10]:
from cases import *

binary_dishwasher, dishwasher_start, start_times = dishwasher(Time_interval,merged_data_winter_case_2,model)
duration_wm, wm_start, binary_wm, start_times_wm = washing_machine(Time_interval,merged_data_winter_case_2,model)
binary_dryer, start_times_dryer, dryer_start = dryer(Time_interval,merged_data_winter_case_2,model, duration_wm, wm_start)
charging_ev, soc_ev, binary_ev = EV_no_feed_in(Time_interval,merged_data_winter_case_2,model)
power_hp = heat_pump(Time_interval,merged_data_winter_case_2,model,max_power_hp)
pv, load, unmet, curtail, pv_maxed_binary, total_demand,penalty_cost, level_bin, levels, penalty_per_level, demand_level= PV_no_feed_in_and_penalty(Time_interval,
                            merged_data_winter_case_2,model,merged_data_winter_case_2,power_dishwasher,binary_dishwasher
                            ,power_wm,binary_wm,power_dryer,binary_dryer,charging_ev,inflexible_demand,max_power_ev,max_power_hp,power_hp)

### Objective Function

In [11]:
# Electricity cost
electricity_cost = gp.quicksum(price[t] * unmet[t] for t in range(Time_interval))

# Full objective: cost + penalty
model.setObjective(electricity_cost + penalty_cost, GRB.MINIMIZE)

# Optimize
model.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[x86] - Darwin 21.6.0 21H1320)



CPU model: Intel(R) Core(TM) i5-6267U CPU @ 2.90GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 48432 rows, 28220 columns and 138582 nonzeros
Model fingerprint: 0xf9269a13
Variable types: 9408 continuous, 18812 integer (17468 binary)
Coefficient statistics:
  Matrix range     [2e-01, 1e+04]
  Objective range  [2e-04, 2e-01]
  Bounds range     [1e+00, 2e+02]
  RHS range        [5e-05, 1e+04]

MIP start from previous solve produced solution with objective 60.8451 (0.70s)
MIP start from previous solve produced solution with objective 60.7308 (0.71s)
MIP start from previous solve produced solution with objective 60.2321 (1.15s)
MIP start from previous solve produced solution with objective 58.7536 (1.29s)
MIP start from previous solve produced solution with objective 57.0926 (2.53s)
MIP start from previous solve produced solution with objective 56.7117 (2.71s)
MIP start from previous solve produced solution with objective 56.5388 (2.94

### Save Results

In [13]:
# Prepare results DataFrame
results_df = merged_data_winter_case_2.copy()
results_df['Total_Demand'] = [total_demand[t].getValue() for t in range(len(results_df))]
results_df['Dishwasher_Start'] = [dishwasher_start[t].X if t in start_times else 0 for t in range(len(results_df))]
results_df['Dishwasher_On'] = [binary_dishwasher[t].X for t in range(len(results_df))]
results_df['Washing_Machine_Start'] = [wm_start[t].X if t in start_times_wm else 0 for t in range(len(results_df))]
results_df['Washing_Machine_On'] = [binary_wm[t].X for t in range(len(results_df))]
results_df['Dryer_Start'] = [dryer_start[t].X if t in start_times_dryer else 0 for t in range(len(results_df))]
results_df['Dryer_On'] = [binary_dryer[t].X for t in range(len(results_df))]
results_df['EV_SOC'] = [soc_ev[t].X for t in range(len(results_df))]
results_df['EV_Charging'] = [charging_ev[t].X for t in range(len(results_df))]
results_df['EV_On'] = [binary_ev[t].X for t in range(len(results_df))]
results_df['Unmet'] = [unmet[t].X for t in range(len(results_df))]
results_df['Demand_Level'] = [demand_level[t].X for t in range(len(results_df))]
results_df['Penalty'] = [
    sum(
        penalty_per_level[i] * level_bin[t][i].X
        for i in range(len(penalty_per_level))
    )
    for t in range(len(results_df))
]
results_df['Electricity_Cost'] = [
    price[t] * unmet[t].X for t in range(len(results_df))
]
results_df['Total_Price'] = [
    results_df['Electricity_Cost'][t] + results_df['Penalty'][t]
    for t in range(len(results_df))
]

# Save to CSV
results_csv_path = os.path.join(project_root, "results_case_2_winter.csv")
results_df.to_csv(results_csv_path, index=False)
print(f"Results saved to {results_csv_path}")

Results saved to /Users/simonbernet/Desktop/Optimization in Energy Systems/Optimization Project/optimization_project/results_case_2_winter.csv


# Case 2 Winter 
## PV Generation without feed-in
### Loading Data